In [1]:
from neuron import h, units, rxd, load_mechanisms
import neuron
import matplotlib
from matplotlib import pyplot as plt
h.load_file("stdrun.hoc")
%matplotlib inline
from itertools import product
import pandas as pd
import os
import numpy as np
import math

--No graphics will be displayed.


In [2]:
class Islet:
    def __init__(self, id, prob_alpha, prob_beta, islet_radius, num_cells):
        """Initialize islet instance"""
        # path to initialization file, mechanisms, output, and generation identifier
        self._id = id
        self.islet_radius = islet_radius
        self.num_cells = num_cells
        self.prob_alpha = prob_alpha
        self.prob_beta = prob_beta
        self.num_betas = math.floor(self.num_cells * self.prob_beta)
        self.num_alphas = math.ceil(self.num_cells * self.prob_alpha)
        self.num_deltas = self.num_cells - self.num_betas - self.num_alphas

    def __repr__(self):
        return 'Islet[{}]'.format(self._id)

    def spatial_setup(self):
        nx, ny, nz = (10,10,10)
        # The below statements take a line from -radius to radius and chops it up into n segments
        x_coords = np.linspace(-self.islet_radius, self.islet_radius, nx)
        y_coords = np.linspace(-self.islet_radius, self.islet_radius, ny)
        z_coords = np.linspace(-self.islet_radius, self.islet_radius, nz)
        self.locations = []
        count = 0
        while count < self.num_cells:
            # Select a random value for x, y, and z coordinates
            x,y,z = np.random.choice(x_coords, 1), np.random.choice(y_coords, 1), np.random.choice(z_coords, 1)
            # Check if the point is within the islet sphere
            if math.sqrt(x**2 + y**2 + z**2) < self.islet_radius:
                # Check if random coordinates chosen are unique so not two cells have the same location
                if any(location == [x[0],y[0],z[0]] for location in self.locations): 
                    continue
                else:
                    # If location unique, append it to the locations list
                    self.locations.append((x[0],y[0],z[0]))
                    count+=1

    def populate_cells(self):
        # Create cell dictionary containing all the sections in the islet
       self.cells = {}
       # Create the sections, which can be accessed by their name in the cells dictionary
       for i in range(self.num_alphas):
            self.cells["A_" + str(i)] = h.Section(name = "A_" + str(i))
       for i in range(self.num_betas):
            self.cells["B_" + str(i)] = h.Section(name = "B_" + str(i))
       for i in range(self.num_deltas):
            self.cells["D_" + str(i)] = h.Section(name = "D_" + str(i))

    def set_cell_locations(self):
        # Separate the cells dictionary into lists for each cell type
        # This is accomplished by using the search criteria of if "A", "B", or "D"
        # is in cell name
        alpha_sections = [val for key, val in self.cells.items() if "A" in key]
        beta_sections = [val for key, val in self.cells.items() if "B" in key]
        delta_sections = [val for key, val in self.cells.items() if "D" in key]
        count = 0
        for i in range(self.num_cells):
            while count < self.num_alphas:
                print("Setting alpha cell locations")
                alpha_sections[i].pt3dclear()
                alpha_sections[i].pt3dadd(self.locations[i][0], self.locations[i][1], self.locations[i][2], 7)
                alpha_sections[i].pt3dadd(self.locations[i][0] + 7, self.locations[i][1], self.locations[i][2], 7)
                count+=1
            while self.num_alphas <= count < self.num_alphas + self.num_betas:
                print("Setting beta cell locations")
                beta_sections[i].pt3dclear()
                beta_sections[i].pt3dadd(self.locations[i][0], self.locations[i][1], self.locations[i][2], 15.5)
                beta_sections[i].pt3dadd(self.locations[i][0] + 15.5, self.locations[i][1], self.locations[i][2], 15.5)
                count+=1
            while self.num_alphas + self.num_betas <= count < self.num_cells:
                print("Setting delta cell locations")
                delta_sections[i].pt3dclear()
                delta_sections[i].pt3dadd(self.locations[i][0], self.locations[i][1], self.locations[i][2], 7)
                delta_sections[i].pt3dadd(self.locations[i][0] + 7, self.locations[i][1], self.locations[i][2], 7)   
                count+=1            

In [3]:
def create_dist_matrix(islet_name):
    # Create a numpy array containing the distance between each cell
    dist_matrix = np.array([math.dist(location_1, location_2) for location_1 in islet_name.locations for location_2 in islet_name.locations])
    # Reshape the array to be nxn where each row corresponds to a single cell and the 
    # distances between itself and all other cells
    dist_matrix = np.reshape(dist_matrix, (islet_name.num_cells, islet_name.num_cells))
    # Add the identity matrix so that the diagonal values are 1
    dist_matrix = dist_matrix + np.identity(islet_name.num_cells)
    return dist_matrix

In [4]:
test_islet = Islet(1,0.3,0.5,1,3)
test_islet.spatial_setup()
test_islet.populate_cells()
test_islet.set_cell_locations()

Setting alpha cell locations
Setting beta cell locations
Setting delta cell locations


In [5]:
create_dist_matrix(test_islet)

array([[1.        , 0.66666667, 0.49690399],
       [0.66666667, 1.        , 1.08866211],
       [0.49690399, 1.08866211, 1.        ]])

In [5]:
test_islet.cells["B_0"].L

15.5

In [31]:
# Cell classes aren't really worth what we are doing.

# class Alpha(Islet):
#     def __init__(self, id, x, y, z):
#         self._id = id
#         self.cell = h.Section(name = "A" + str(id), cell = self)
#         self.cell.pt3dclear()
#         self.cell.pt3dadd(x,y,z,7)
#         self.cell.pt3dadd(x+7, y,z,7)
#     def __repr__(self):
#         return 'Alpha[{}]'.format(self._id)

# class Beta(Islet):
#     def __init__(self, id, x, y, z):
#         self._id = id
#         self.cell = h.Section()
#         self.cell.pt3dclear()
#         self.cell.pt3dadd(x,y,z,15.5)
#         self.cell.pt3dadd(x+15.5, y,z,15.5)
#     def __repr__(self):
#         return 'Beta[{}]'.format(self._id)

# class Delta(Islet):
#     def __init__(self, id, x, y, z):
#         self._id = id
#         self.cell = h.Section()
#         self.cell.pt3dclear()
#         self.cell.pt3dadd(x,y,z,7)
#         self.cell.pt3dadd(x+7, y,z,7)
#     def __repr__(self):
#         return 'Delta[{}]'.format(self._id)